## Data Preparation



In [50]:
#%pip install pymysql
#%pip install pandas
#%pip install openpyxl
#%pip install numpy

# installing libraries to work with data

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [117]:
import pymysql
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

# loading libraries to work with data

In [4]:
host = os.getenv("EDREAMS_HOST")
port = os.getenv("EDREAMS_PORT")
username = os.getenv("EDREAMS_USERNAME")
userpass = os.getenv("EDREAMS_USERPASS")
database = os.getenv("EDREAMS_DEFAULT_DB")

# settings as constant variables

In [29]:
def connect():
    connection = pymysql.connect(
        host=host,
        user=username,
        password=userpass
    )
    cur = connection.cursor()
    cur.execute("show databases")
    output = cur.fetchall()
    print(output)
    connection.close()
    
connect()

#testing the database connection on RDS AWS - open only to my internal IP so no connection to the outside world

(('edreams',), ('information_schema',), ('mysql',), ('performance_schema',), ('sys',))


In [9]:
martech_file = "sql_data_martech.xlsx"
funnel_table = pd.DataFrame(pd.read_excel(martech_file, sheet_name=0))
funnel_table = funnel_table.replace(np.nan, 'empty')
booking_table = pd.DataFrame(pd.read_excel(martech_file, sheet_name=1))
booking_table = booking_table.replace(np.nan, 'empty')
#funnel_table
#funnel_table.dtypes

booking_table
booking_table.dtypes

booking_table

# loading up the excel sheets into pandas to load up into the mysql table

,transactionid,product,revenue
0,1573320,Flight,747.01
1,9510159,Car,669.22
2,8029373,Flight,1173.59
3,2515973,Flight,1345.44
4,6717417,Car,699.52
...,...,...,...
332,8913717,Flight,1074.43
333,9487109,Flight,524.12
334,7260096,Flight,1491.32
335,4043573,Flight,934.29


In [28]:
sql_query_create_db = "CREATE DATABASE edreams"

def create_database(sql_query):
    connection = pymysql.connect(
        host=host,
        user=username,
        password=userpass,
    )
    cur = connection.cursor()
    cur.execute(sql_query)
    connection.close()
    

create_database(sql_query_create_db)
# run once to create a database for edreams

In [12]:
def run_query(sql_query):
    connection = pymysql.connect(
        host=host,
        user=username,
        password=userpass,
        database=database
    )
    cur = connection.cursor()
    cur.execute(sql_query)
    output = cur.fetchall()
    print(output)
    cur.close()
    connection.close()

In [36]:
sql_delete = "DROP TABLE FunnelTable"

run_query(sql_delete)

# sessionID as varchar since I don't need to perform matchematical operations on them
sql_query_funnel = """
CREATE TABLE FunnelTable (
  userid VARCHAR(255) NOT NULL,
  sessionid VARCHAR(255) NOT NULL,
  timestamp DATETIME NOT NULL,
  page VARCHAR(255) NOT NULL,
  channel VARCHAR(255) NOT NULL,
  device VARCHAR(255) NOT NULL,
  browser VARCHAR(255) NOT NULL,
  country VARCHAR(255) NOT NULL,
  transactionid VARCHAR(255) NOT NULL
);
"""

run_query(sql_query_funnel)

In [34]:
sql_query_booking = """
CREATE TABLE BookingTable (
  transactionid VARCHAR(255) NOT NULL,
  product VARCHAR(255) NOT NULL,
  revenue FLOAT NOT NULL
);
"""

run_query(sql_query_booking)

In [39]:
run_query("SHOW TABLES")

(('BookingTable',), ('FunnelTable',))


In [5]:
def insert_funnel(df):
    connection = pymysql.connect(
        host=host,
        user=username,
        password=userpass,
        database=database
    )
    cur = connection.cursor()
    for index, row in df.iterrows():    
        userid = row['userid']
        sessionid = row['sessionid']
        timestamp = row['timestamp']
        page = row['page']
        channel = row['channel']
        device = row['device']
        browser = row['browser']
        country = row['country']
        transactionid = row['transactionid']

        sql_query = "INSERT INTO FunnelTable (userid, sessionid, timestamp, page, channel, device, browser, country, transactionid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cur.execute(sql_query, (userid, sessionid, timestamp, page, channel, device, browser, country, transactionid))
    connection.commit()
    cur.close()
    connection.close()
    
insert_funnel(funnel_table)

# inserting data into the FunnelTable for further exploratory analysis

In [11]:
def insert_booking(df):
    connection = pymysql.connect(
        host=host,
        user=username,
        password=userpass,
        database=database
    )
    cur = connection.cursor()
    for index, row in df.iterrows():    
        transactionid = row['transactionid']
        product = row['product']
        revenue = row['revenue']

        sql_query = "INSERT INTO BookingTable (transactionid, product, revenue) VALUES (%s, %s, %s)"
        cur.execute(sql_query, (transactionid, product, revenue))
    connection.commit()
    cur.close()
    connection.close()
    
insert_booking(booking_table)

# inserting data into the BookingTable for further exploratory analysis

## Instructions

For each of the questions above, provide a SQL query and its result that answers them. Please,
deliver the queries in text format and, regarding their results, you could do it in the same way in a
document or by pasting screenshots of the SQL administration tool that you used.

In [32]:
def query_to_df(sql_query):
    connection = pymysql.connect(
        host=host,
        user=username,
        password=userpass,
        database=database
    )
    df = pd.read_sql(sql_query, connection)
    connection.close()
    return df

In [120]:
# 1. What is the total revenue generated per each country AND device the 2nd of November, 2019?

sql_query_1 = """
SELECT country, device, SUM(revenue) AS total_revenue
FROM FunnelTable
INNER JOIN BookingTable
ON FunnelTable.transactionid = BookingTable.transactionid
WHERE timestamp BETWEEN '2019-11-02 00:00:00' AND '2019-11-02 23:59:59'
GROUP BY country, device
ORDER BY total_revenue DESC;
"""

df1 = query_to_df(sql_query_1)
df1

,country,device,total_revenue
0,UK,mobile,25767.049805
1,Germany,mobile,24799.939896
2,France,mobile,23315.939789
3,Italy,mobile,19342.399933
4,Spain,mobile,17974.279968
5,Spain,desktop,15619.690186
6,Portugal,mobile,12133.630188
7,Italy,desktop,11989.929977
8,France,desktop,10902.849976
9,Portugal,desktop,10726.340027


In [121]:
# 2. What country has the most page views per session on average?

sql_query_2 = """
WITH session_pageviews AS (
  SELECT country, sessionid, COUNT(page) AS pageviews
  FROM FunnelTable
  GROUP BY country, sessionid
)
SELECT country, AVG(pageviews) AS avg_pageviews
FROM session_pageviews
GROUP BY country
ORDER BY avg_pageviews DESC
"""

df2 = query_to_df(sql_query_2)
print(df2.iloc[0].country, "has the most page views per session on average")

UK has the most page views per session on average


In [122]:
# 3. How many sessions have seen the “home” page AND the “payment” page?


sql_query_3 = """
SELECT COUNT(DISTINCT sessionid) 
FROM FunnelTable 
WHERE page IN ('home', 'payment') 
GROUP BY sessionid;
"""

df3 = query_to_df(sql_query_3)
print(len(df3), "sessions have seen the home and payment page")

2729 sessions have seen the home and payment page


In [133]:
# 4. We define “continuance rate” as the percentage of sessions that have progressed from
# payment to confirmation, i.e. (sessions that have seen confirmation) / (sessions that have
# seen payment). Compute it by country AND device.

sql_query_4_bk = """
WITH session_status AS (
  SELECT country, device, sessionid, 
         MAX(CASE WHEN page = 'payment' THEN 1 ELSE 0 END) AS payment,
         MAX(CASE WHEN page = 'confirmation' THEN 1 ELSE 0 END) AS confirmation
  FROM FunnelTable
  GROUP BY country, device, sessionid
)
SELECT country, device, 
       SUM(CASE WHEN payment = 1 AND confirmation = 1 THEN 1 ELSE 0 END) / 
       SUM(CASE WHEN payment = 1 THEN 1 ELSE 0 END) * 100 AS continuance_rate
FROM session_status
GROUP BY country, device
ORDER BY continuance_rate DESC;
"""

sql_query_4 = """
SELECT country, device, COUNT(DISTINCT CASE WHEN page = 'payment' THEN sessionid END) AS sessions_with_payment, 
       COUNT(DISTINCT CASE WHEN page = 'confirmation' THEN sessionid END) AS sessions_with_confirmation, 
       COUNT(DISTINCT CASE WHEN page = 'confirmation' THEN sessionid END) / COUNT(DISTINCT CASE WHEN page = 'payment' THEN sessionid END) AS continuance_rate
FROM FunnelTable
GROUP BY country, device
ORDER BY continuance_rate DESC;
"""

df4 = query_to_df(sql_query_4)
df4

,country,device,sessions_with_payment,sessions_with_confirmation,continuance_rate
0,Portugal,tablet,12,7,0.5833
1,Spain,mobile,62,35,0.5645
2,France,mobile,80,40,0.5000
3,Italy,desktop,40,20,0.5000
4,Spain,desktop,46,23,0.5000
5,Italy,mobile,64,31,0.4844
6,Germany,desktop,30,14,0.4667
7,Germany,mobile,82,36,0.4390
8,UK,mobile,87,38,0.4368
9,Portugal,desktop,40,17,0.4250


In [145]:
# 5. We define “landing page” as the first page in a session. What is the most abundant landing page?

sql_query_5 = """
SELECT page, COUNT(FunnelTable.sessionid) AS session_count
FROM (
  SELECT sessionid, MIN(timestamp) AS first_timestamp
  FROM FunnelTable
  GROUP BY sessionid
) AS first_sessions
JOIN FunnelTable ON first_sessions.sessionid = FunnelTable.sessionid
  AND first_sessions.first_timestamp = FunnelTable.timestamp
GROUP BY page
ORDER BY session_count DESC
"""

df5 = query_to_df(sql_query_5)
df5

,page,session_count
0,home,2408
1,black friday,538
2,christmas offers,458


In [152]:
# 6. We define conversion rate as (total transactions) / (total sessions). What is the conversion rate per landing page AND device?

sql_query_6 = """
SELECT page,
       device,
       COUNT(DISTINCT transactionid) / COUNT(DISTINCT sessionid) AS conversion_rate
FROM FunnelTable
GROUP BY page, device
ORDER BY conversion_rate DESC;
"""

df6 = query_to_df(sql_query_6)
df6

,page,device,conversion_rate
0,confirmation,mobile,1.0145
1,confirmation,tablet,1.0000
2,confirmation,desktop,1.0000
3,upsell,tablet,0.0227
4,christmas offers,tablet,0.0222
5,black friday,tablet,0.0192
6,payment,tablet,0.0172
7,details,tablet,0.0085
8,upsell,desktop,0.0065
9,christmas offers,desktop,0.0056


In [162]:
# 7. We define “Exit rate” as the percentage of times a page was the last one in a session, out of
# all the times the page was viewed. What is the exit rate of “ results”?

sql_query_7 = """
WITH page_session AS (
  SELECT sessionid, page, 
         ROW_NUMBER() OVER (PARTITION BY sessionid ORDER BY timestamp DESC) AS row_num 
  FROM FunnelTable
),
exit_rate AS (
  SELECT page, COUNT(*) AS exits 
  FROM page_session 
  WHERE row_num = 1 
  GROUP BY page
),
total_views AS (
  SELECT page, COUNT(*) AS total_views 
  FROM FunnelTable 
  GROUP BY page
)
SELECT exit_rate.page, (exit_rate.exits / total_views.total_views) * 100 AS exit_rate 
FROM exit_rate 
JOIN total_views ON exit_rate.page = total_views.page 
WHERE exit_rate.page = 'results';
"""


df7 = query_to_df(sql_query_7)
df7

,page,exit_rate
0,results,29.8597
